In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from tkinter import simpledialog
from tkinter import filedialog
from geopy.geocoders import Nominatim, ArcGIS,MapBox
import geopandas as gpd
from functools import  partial

%matplotlib inline

In [35]:
dirDeArchivo = filedialog.askopenfilename(title="Abrir Archivo excel")
TipoArchivo = simpledialog.askstring(title="Que tipo de Archvo es ? Medico o Institución",prompt="ingresa la palabra Medico o Institucion",initialvalue="MEDICO").upper()
print(TipoArchivo)

verExcelOriginal= pd.read_excel(dirDeArchivo,0,header=1)
verExcelOriginal.head()

MEDICO


,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Cedula,Tel1,Tel2,Celular,Email,Rango Edad,Da Consulta en Hosp,Perfil,Fecha_Alta,Fecha_Mod
0,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{6C700081-38B6-47BB-A2A3-80027DE2BEE5},{08893F58-1925-46AD-B982-13D8C5D4C6B7},HOSPITALES,MEDICO,AGUILERA,ROMERO,PEDRO MAX,TAMAULIPAS 255 Y OAXACA,...,10825666,5557782600,NaN,NaN,NaN,28.0,SI,NaN,NaT,2019-02-19
1,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{B602CE7E-76EB-4B0F-BB3B-B7967E4F5C35},{2422BB5E-D8D1-4B74-B718-E90B9CBE2C24},CONSULTORIOS,MEDICO,AGUILERA,SAUCEDO,GISELA,AV GUSTAVO BAZ MZ 70 LTE 10 Y GOB F MUEGIA,...,3395130,5561415107,NaN,NaN,giselaguilera_76@hotmail.com,44.0,SI,NaN,NaT,2019-02-06
2,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{3C55C317-29C7-411A-9779-89FB8C6B6CF2},{F0D1F7BB-D006-4502-9109-7B470D61B92D},HOSPITALES,MEDICO,ALARCON,FLORES,MARIBEL,ROSAS 403 Y AV. SANTA TERESA,...,3243250,5558821912,NaN,NaN,NaN,48.0,NO,NaN,NaT,2019-02-25
3,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{58088CB9-F967-4F45-A012-E015398B1C44},{299CAF8C-706D-40CF-9DE8-03B0FC9415E8},CONSULTORIOS,MEDICO,ALBARRAN,GUERRERO,ALICIA,CARRETERA REYES ZUMPANGO KM 38,...,4512455,7711435357,NaN,NaN,medica_gya@yaoo.com.mx,NaN,NO,NaN,2023-08-01,2023-08-30
4,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{2C9AC237-F4E2-48CF-A4A1-430B3AABD1C6},{2FE0FAEB-5069-4D36-BC3A-CACD9E61E313},CONSULTORIOS,MEDICO,ALCANTARA,CHAVEZ,BEATRIZ,CONS. IGNACIO RAYON LOCAL 3 4 Y JARDINES DE MO...,...,2679562,5580844035,NaN,NaN,dra.alcantara@live.com.mx,NaN,NO,NaN,2021-10-14,2022-03-06


In [36]:
verExcelOriginal.columns

verExcel = verExcelOriginal.rename(columns={'Num Ext': 'Num_Ext'})

In [45]:
verExcel['Direccion']= verExcel["Calle"]
verExcel['Col']=verExcel["Colonia"].str[7:]
verExcel['Ciudad']=verExcel["Poblacion"]
verExcel["Pais"]="Mexico"

verExcel.head()

,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Email,Rango Edad,Da Consulta en Hosp,Perfil,Fecha_Alta,Fecha_Mod,Direccion,Col,Ciudad,Pais
0,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{6C700081-38B6-47BB-A2A3-80027DE2BEE5},{08893F58-1925-46AD-B982-13D8C5D4C6B7},HOSPITALES,MEDICO,AGUILERA,ROMERO,PEDRO MAX,TAMAULIPAS 255 Y OAXACA,...,NaN,28.0,SI,NaN,NaT,2019-02-19,TAMAULIPAS 255 Y OAXACA,SANTA MARIA TULPETLAC,ECATEPEC DE MORELOS,Mexico
1,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{B602CE7E-76EB-4B0F-BB3B-B7967E4F5C35},{2422BB5E-D8D1-4B74-B718-E90B9CBE2C24},CONSULTORIOS,MEDICO,AGUILERA,SAUCEDO,GISELA,AV GUSTAVO BAZ MZ 70 LTE 10 Y GOB F MUEGIA,...,giselaguilera_76@hotmail.com,44.0,SI,NaN,NaT,2019-02-06,AV GUSTAVO BAZ MZ 70 LTE 10 Y GOB F MUEGIA,GRANJAS VALLE DE GUADALUPE SECCION B,ECATEPEC DE MORELOS,Mexico
2,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{3C55C317-29C7-411A-9779-89FB8C6B6CF2},{F0D1F7BB-D006-4502-9109-7B470D61B92D},HOSPITALES,MEDICO,ALARCON,FLORES,MARIBEL,ROSAS 403 Y AV. SANTA TERESA,...,NaN,48.0,NO,NaN,NaT,2019-02-25,ROSAS 403 Y AV. SANTA TERESA,GUSTAVO DIAZ ORDAZ,ECATEPEC DE MORELOS,Mexico
3,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{58088CB9-F967-4F45-A012-E015398B1C44},{299CAF8C-706D-40CF-9DE8-03B0FC9415E8},CONSULTORIOS,MEDICO,ALBARRAN,GUERRERO,ALICIA,CARRETERA REYES ZUMPANGO KM 38,...,medica_gya@yaoo.com.mx,NaN,NO,NaN,2023-08-01,2023-08-30,CARRETERA REYES ZUMPANGO KM 38,SAN LUCAS XOLOX,TECAMAC,Mexico
4,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{2C9AC237-F4E2-48CF-A4A1-430B3AABD1C6},{2FE0FAEB-5069-4D36-BC3A-CACD9E61E313},CONSULTORIOS,MEDICO,ALCANTARA,CHAVEZ,BEATRIZ,CONS. IGNACIO RAYON LOCAL 3 4 Y JARDINES DE MO...,...,dra.alcantara@live.com.mx,NaN,NO,NaN,2021-10-14,2022-03-06,CONS. IGNACIO RAYON LOCAL 3 4 Y JARDINES DE MO...,JARDINES DE MORELOS SECCION LAGOS,ECATEPEC DE MORELOS,Mexico


In [46]:
def concatena_campos(row):
    return f"{row['Direccion']},{row['Col']},{row['Ciudad']},{row['Estado']},{row['Pais']}"

In [47]:
verExcel["noGeo"]= verExcel.apply(concatena_campos,axis=1)
verExcel.head()

verExcel.to_excel("archivodePrueba.xlsx")

In [19]:
verExcel_for_osm = verExcel.copy()
verExcel_for_mapbox = verExcel.copy()
verExcel_for_arcgis = verExcel.copy()

geocodificacion completa con la API de OSM

In [20]:
geolocator =Nominatim(user_agent="jorgemc",timeout=5)
geocode = partial(geolocator.geocode,language = "es")

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    



In [ ]:
verExcel_for_osm[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_osm

Geolocalizando con ArcGis

In [24]:
api_arcgis = "AAPKe017698084d742049005e493d4bce9ddY45j51f68jQMWf7cA43OuO3zs160_aYigr1xskjzOzm23Y8n3ijvKA5DlYurkuOq"
geolocator = ArcGIS(auth_domain=api_arcgis)

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    

In [25]:
verExcel_for_arcgis[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_arcgis

,Linea,Representante,Cod Inst,Tipo,Sub_Tipo,Nombre,Direccion,Col,Ciudad,Estado,...,Estatus,Colonia,Categoria,Ubicacion,Email,Fecha_Alta,Fecha_Mod_Baja,noGeo,latitude,longitude
0,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{C55C9107-61F9-47F8-A407-BBA35BADDC85},HOSPITAL,PRIVADO,CENTRO DE REHABILITACION,BRASIL S/N Y AV. SANTA TERESA,NUEVO PASEO DE SAN AGUSTIN,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55130 - NUEVO PASEO DE SAN AGUSTIN,NaN,NaN,NaN,NaN,NaN,"BRASIL S/N Y AV. SANTA TERESA,NUEVO PASEO DE S...",19.524077,-99.040962
1,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{2A696C60-7BB8-44EC-A82A-3D214C86BC0E},HOSPITAL,PRIVADO,CLINICA ALIANZA,AV CENTRAL 43 Y VALLE DE LIMPORIO,VALLE DE ARAGON 3RA SECCION PONIENTE,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55280 - VALLE DE ARAGON 3RA SECCION PONIENTE,NaN,NaN,NaN,NaN,NaN,"AV CENTRAL 43 Y VALLE DE LIMPORIO,VALLE DE ARA...",19.553051,-99.019094
2,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{E8963C31-7D77-4D5C-8795-E9AB5E80A30A},HOSPITAL,PRIVADO,CLINICA DEL CONSUELO,AV JARDINES DE MORELOS 203 Y MIGUEL HIDALGO,JARDINES DE MORELOS SECCION BOSQUES,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55070 - JARDINES DE MORELOS SECCION BOSQUES,NaN,NaN,NaN,NaN,NaN,"AV JARDINES DE MORELOS 203 Y MIGUEL HIDALGO,JA...",19.595595,-99.004973
3,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{6D0FE50C-8254-42B0-8073-CF0191148A71},HOSPITAL,PRIVADO,CLINICA DEL ROCIO,IGNACIO ALLENDE 23 Y ESQ CEREZO,LA CRUZ DE SAN CRISTOBAL,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55087 - LA CRUZ DE SAN CRISTOBAL,NaN,NaN,NaN,NaN,45146.0,"IGNACIO ALLENDE 23 Y ESQ CEREZO,LA CRUZ DE SAN...",19.597563,-99.047415
4,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{C5010CE0-EB99-4A3C-8369-A9D3DC9330D3},HOSPITAL,PRIVADO,CLINICA DEL ROCIO,IGNACIO ALLENDE 23 Y ESQ CEREZO,SAN CRISTOBAL CENTRO,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55000 - SAN CRISTOBAL CENTRO,NaN,NaN,NaN,45146.0,45146.0,"IGNACIO ALLENDE 23 Y ESQ CEREZO,SAN CRISTOBAL ...",19.597563,-99.047415
5,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{7273F523-4D7D-4DE4-AED3-49E63677CAFA},HOSPITAL,PRIVADO,CLINICA GUARNEROS,AV GOB LIC IGNACIO PICHARDO PAGAZA 70 Y VALLE ...,FUENTES DE ARAGON,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55248 - FUENTES DE ARAGON,NaN,NaN,NaN,NaN,NaN,AV GOB LIC IGNACIO PICHARDO PAGAZA 70 Y VALLE ...,19.513491,-99.031427
6,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{56A25D3D-9B3C-4169-858A-FF41A025DB8E},HOSPITAL,PRIVADO,CLINICA MEDICO QUIRURGICA,BLVD QUETZALCOATL MZ 139 LTE 58 Y FUEGO NUEVO,LA FLORIDA (CIUDAD AZTECA),ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55129 - LA FLORIDA (CIUDAD AZTECA),NaN,NaN,NaN,NaN,NaN,"BLVD QUETZALCOATL MZ 139 LTE 58 Y FUEGO NUEVO,...",19.542631,-99.019156
7,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{2D5001A5-078F-4585-9D90-B5ABCC346FF8},HOSPITAL,PRIVADO,CLINICA MEDICO QUIRURGICAS 24 HORAS OJO DE AGUA,PASEO DE LOS PENSAMIENTOS MZ 62 Y PASEO DE LA ...,OJO DE AGUA,TECAMAC,MEXICO,...,ACTIVO,55770 - OJO DE AGUA,NaN,NaN,NaN,NaN,NaN,PASEO DE LOS PENSAMIENTOS MZ 62 Y PASEO DE LA ...,19.672477,-99.022021
8,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{F8210D8E-8CCA-4E38-83F3-78165D6F2A86},HOSPITAL,PRIVADO,CLINICA MET,FCO GLEZ BOCA NEGRA 38 Y 5 DE FEBRERO,TECAMAC CENTRO,TECAMAC,MEXICO,...,ACTIVO,55748 - TECAMAC CENTRO,NaN,NaN,NaN,NaN,NaN,"FCO GLEZ BOCA NEGRA 38 Y 5 DE FEBRERO,TECAMAC ...",19.711316,-98.973540
9,RAYERE,P1201 - HERRERA CERVANTES ROBERTO,{412FDBBB-B16B-43BC-A3A8-641DB0B1D7F9},HOSPITAL,PRIVADO,CLINICA NUESTRA SENORA DEL SOCORRO,DIAGONAL SANTA CLARA 3 Y FCO JAVIER MINA,SANTA CLARA COATITLA,ECATEPEC DE MORELOS,MEXICO,...,ACTIVO,55540 - SANTA CLARA COATITLA,NaN,NaN,NaN,NaN,NaN,"DIAGONAL SANTA CLARA 3 Y FCO JAVIER MINA,SANTA...",19.539647,-99.067442


Se creo un Dataframe con geometria y con una Referencia de sistema de Coordenadas.   en este caso tenemos un sistema de coordenasas ITRF 2008 y Geometria de Puntos. 

In [26]:
geo_rawData = gpd.GeoDataFrame(data=verExcel_for_arcgis,geometry=gpd.points_from_xy(verExcel_for_arcgis.longitude,verExcel_for_arcgis.latitude),crs=6363)

Abrir mapa para mostrar datos.

In [ ]:
geo_rawData.explore()

Convertir el archivo a un GeoJson o Un Shapefile

In [29]:
geo_rawData.to_file('Hospitales_Herrera.shp')
geo_rawData.to_file('Hospitales_Herrera.geojson',driver="GeoJSON")

C:\Users\Invmerp01\AppData\Local\Temp\ipykernel_6260\1160993505.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_rawData.to_file('Hospitales_Herrera.shp')
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Representante' to 'Representa'
  ogr_write(
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Fecha_Mod_Baja' to 'Fecha_Mod_'
  ogr_write(
